## Problem Statement


Clearly define the problem statement that your chosen feature aims to address.
Explain the significance of this problem in the context of climate risk management.


This project will improve the model's ability to estimate the price to rebuild a home after flooding by accounting for more complex aspects of discount rate and price changes based on scarcity after a flood.


To obtain a more accurate understanding of cost over time, the discount rate will be broken down to have separate discount rates for the net present value calculation, the housing market, and the cost of labor and materials. Including these values will capture the way that different aspects of cost change over time in different ways; for example, perhaps inflation is higher for labor or materials. These factors will affect estimates of the cost to rebuild a house entirely from scratch. The discount rates, which may have different distributions or be constant, will be decided based on the findings of the literature review.


Additionally, the model will be modified to capture how scarcity caused by a flood affects construction cost by multiplying reconstruction costs by a scarcity factor that will be decided after the literature review. These factors are significant as our current model has the value of the house be static even after 60 years, which is unrealistic. This makes us vulnerable to underestimating the cost to rebuild the house in the future, possibly making low elevations optimal compared to higher elevations. With more accurate construction costs, we should have less of an issue with overestimating the NPV at different elevations.


The components added to the model are illustrated in Figure 1.


<!-- ![housing_cost_pipeline](house_cost_pipeline.png){width=80%} -->






## Selected Feature

Describe the feature you have selected to add to the existing decision-support tool.
Discuss how this feature relates to the problem statement and its potential to improve climate risk assessment.

The two features we've decided to add are additional discount rates in order to model price increase for construction and labor costs as well as a factor that multiplies the cost to rebuild the house in order to model scarcity after a flood event. This will improve the climate risk assessment as it more accurately shows the magnitude of the costs associated with rebuilding the house. Under our previous model without these changes, we are likely to come to an optimal solution that is lower than what the elevation should actually be in the real world. The increased cost to rebuild in the future may make solutions using a higher elevation in year 1 more robust.

# Literature Review

Provide a brief overview of the theoretical background related to your chosen feature.
Cite at least two relevant journal articles to support your approach (see [Quarto docs](https://quarto.org/docs/authoring/footnotes-and-citations.html) for help with citations).
Explain how these articles contribute to the justification of your selected feature.

The majority of articles refer to construction cost increases after natural disasters as “demand surges”. While many factors are theorized to play into demand surges, such as increased labor costs, material costs, etc., there is little consensus of how these factors quantitatively play into the magnitude of the demand surge following a hazard. Despite the lack of formal consensus on the topic, demand surge can still be approximately modeled based on historical data and basing our assumptions from previous research. For example, the paper with the highest claimed demand surge was 20%. From this, we can make assumptions about our boundaries for the demand surge with it ranging from 0-20%. Further research on the topic has also shown there is a correlation between hazard magnitude and labor costs in East Coast states during hurricane season. Interestly, though, there was no significant correlation between the hazard and materials cost. Regardless of that finding, though, this paper shows a possible correlation between the hazard’s magnitude and a factor that plays into demand surge. Using these findings, we can build our model to be a function where the input is the magnitude of the hazard and the output is the demand surge. Finally, previous research articles have included models for demand surge using a linear relationship. Copying this method can complete the function discussed previously, as our relationship between hazard magnitude and demand surge can simply be modeled as a linear relationship too.

# Methodology

## Implementation

You should make your modifications in either the `HouseElevation` or `ParkingGarage` module.
Detail the steps taken to implement the selected feature and integrate it into the decision-support tool.
Include code snippets and explanations where necessary to clarify the implementation process.

The scarcity/demand surge feature will be added by assuming a linear relationship between the damage of the house and the demand surge magnitude. The highest value for a demand surge was found to be around 20%. Thus, if a house was damaged by 50%, the demand surge will be 50% of 20%, which is 10%. This 10% will then be multiplied onto the damage that the house suffered in order to model how much more it would cost to rebuild under scarcity. For the example before, this would be 50% * (1 + 10%) = 55% damage.

## Validation

As we have seen in labs, mistakes are inevitable and can lead to misleading results.
To minimize the risk of errors making their way into final results, it is essential to validate the implemented feature.
Describe the validation techniques used to ensure the accuracy and reliability of your implemented feature.
Discuss any challenges faced during the validation process and how they were addressed.

# Results

Present the results obtained from the enhanced decision-support tool.
Use tables, figures, and visualizations to clearly communicate the outcomes.
Provide sufficient detail to demonstrate how the implemented feature addresses the problem statement.
Use the `#| output: false` and/or `#| echo: false` tags to hide code output and code cells in the final report except where showing the output (e.g.g, a plot) or the code (e.g., how you are sampling SOWs) adds value to the discussion.
You may have multiple subsections of results, which you can create using `##`.

# Conclusions

## Discussion

Analyze the implications of your results for climate risk management.
Consider the context of the class themes and discuss how your findings contribute to the understanding of climate risk assessment.
Identify any limitations of your approach and suggest potential improvements for future work.

## Conclusions

Summarize the key findings of your project and reiterate the significance of your implemented feature in addressing the problem statement.
Discuss the broader implications of your work for climate risk management and the potential for further research in this area.

# References

:::{#refs}
:::


In [ ]:
using CSV
using DataFrames
using DataFramesMeta
using Distributions
using LaTeXStrings
using Metaheuristics
using Plots
using Random
using Unitful

Plots.default(; margin=5Plots.mm)

We also load our local package as in lab 5.


In [ ]:
using Revise
using HouseElevation

# States of the world

We begin by defining the variables that don't change from one SOW to the next.
We load these into the `ModelParams`.


In [ ]:
#| output: false
house = let
    haz_fl_dept = CSV.read("data/haz_fl_dept.csv", DataFrame) # read in the file
    desc = "one story, Contents, fresh water, short duration"
    row = @rsubset(haz_fl_dept, :Description == desc)[1, :] # select the row I want
    area = 500u"ft^2"
    height_above_gauge = 12u"ft"
    House(row; area=area, height_above_gauge=height_above_gauge, value_usd=250_000)
end

p = ModelParams(; house=house, years=2024:2083)

Next we define how we will sample the states of the world.


In [ ]:
#| output: false
slr_scenarios = let
    df = CSV.read("data/slr_oddo.csv", DataFrame)
    [Oddo17SLR(a, b, c, tstar, cstar) for (a, b, c, tstar, cstar) in eachrow(df)]
end

function draw_surge_distribution()
    μ = rand(Normal(5, 1))
    σ = rand(Exponential(1.25))
    ξ = rand(Normal(0.1, 0.05))
    return GeneralizedExtremeValue(μ, σ, ξ)
end

function draw_discount_rate()
    return rand(Normal(0.05, 0.03))
end

function house_discount_rate()
    return 0.045
end

function draw_sow()
    slr = rand(slr_scenarios)
    surge_params = draw_surge_distribution()
    discount_rate = draw_discount_rate()
    house_discount = house_discount_rate()
    return SOW(slr, surge_params, discount_rate, house_discount)
end

Finally we can sample the SOWs


In [ ]:
#| output: false
Random.seed!(421521)
N_SOW = 10_000
N_SOW_opt = 10 # to start
sows = [draw_sow() for _ in 1:N_SOW]
sows_opt = first(sows, N_SOW_opt)

# Optimization

## Bounds

We have a single decision variable, the height of the house above the ground.
This can be any real number between 0 and 14 feet.


In [ ]:
bounds = boxconstraints(; lb=[0.0], ub=[14.0])

## Objective function

We next need an objective function.
Recall that we want to _maximize_ NPV, but the optimization package we are using is set up to _minimize_.

## New model


In [ ]:
function objective_function(Δh::Vector{Float64})
    a = Action(Δh[1])
    npvs = [run_sim(a, sow, p, false) for sow in sows_opt]
    return -mean(npvs)
end

## Running

We can throw this straight into the `optimize` function:


In [ ]:
result = optimize(objective_function, bounds)

We can view the minimum of the objective function with


In [ ]:
minimum(result)

and the value of the decision variable that achieves that minimum with:


In [ ]:
minimizer(result)

In [ ]:
#| output: false
elevations_try = 0:0.5:14
actions_try = Action.(elevations_try)

N_more = 500
npvs_moore = [
    mean([run_sim(a, sow, p, false) for sow in first(sows, N_more)]) for a in actions_try
]

and plot


In [ ]:
plot(
    elevations_try,
    npvs_moore ./ 1000;
    xlabel="Elevation [ft]",
    ylabel="NPV [1000 USD]",
    label="First $(N_more) SOWs",
    marker=:circle,
)

In [ ]:
function objective_function(Δh::Vector{Float64})
    a = Action(Δh[1])
    npvs = [run_sim_old(a, sow, p, false) for sow in sows_opt]
    return -mean(npvs)
end

## Running

We can throw this straight into the `optimize` function:


In [ ]:
result = optimize(objective_function, bounds)

We can view the minimum of the objective function with


In [ ]:
minimum(result)

and the value of the decision variable that achieves that minimum with:


In [ ]:
minimizer(result)

In [ ]:
#| output: false
elevations_try = 0:0.5:14
actions_try = Action.(elevations_try)

N_more = 500
npvs_moore = [
    mean([run_sim_old(a, sow, p, false) for sow in first(sows, N_more)]) for a in actions_try
]

and plot


In [ ]:
plot(
    elevations_try,
    npvs_moore ./ 1000;
    xlabel="Elevation [ft]",
    ylabel="NPV [1000 USD]",
    label="First $(N_more) SOWs",
    marker=:circle,
)

### Validation

To confirm that the implementation of the housing discount rate is taking place as expected, we look at one randomly sampled state of the world and a select few years. The expected actual damages computed as a function of the sampled sea level rise, sampled storm surge distribution, and housing characteristics that do not change are shown. 

The the expected actual damages calculated based on one possible state of the world and the same fixed house characteristics used in the full implementation are shown with one action option of elevating the house 1 foot in a limited number of possible years. The housing multiplier for each year, which takes into account a housing "discount rate" that describes the housing value appreciation rate, estimated here as 4.5%, is also shown. Because the housing value increased by 4.5% each year, the multiplier is continuously compounding. Next, the expected actual damage calculated for each year for a house of the given value, $250,000, is shown. These values confirm that the calculations are taking place as intended.


In [ ]:
#| code-overflow: wrap
run_sim(Action.(1.0), first(sows), ModelParams(; house=house, years=[2024, 2030]), true)

For example, for an expected actual damage share of 0.011587508736959039 in 2030,

$EAD = 0.011587508736959039 * 250000 * (1.045)^5 = 3772.4876436159875$.

To confirm that the housing multipliers decline as intended after 25 and 75 years:


In [ ]:
#| code-overflow: wrap
run_sim(Action.(1.0), first(sows), ModelParams(; house=house, years=2024:2065), true)
ModelParams(; house=house, years=2024:2083)